In [14]:
import pandas_datareader.data as web
import datetime

start = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2022, 7, 1)
df = web.DataReader('GOOGL', 'stooq', start, end)
print(df)

                 Open      High        Low      Close       Volume
Date                                                              
2022-07-01  107.93300  109.2500  106.73000  108.73800   35475660.0
2022-06-30  110.00000  110.8880  106.75000  108.96300   43169060.0
2022-06-29  111.55200  113.1600  110.87300  111.70200   24716820.0
2022-06-28  115.80300  117.3100  111.84800  112.00800   35880960.0
2022-06-27  118.27300  118.5800  115.18000  115.83400   36420760.0
...               ...       ...        ...        ...          ...
2004-08-25    2.62400    2.7000    2.59700    2.65000  183956000.0
2004-08-24    2.78100    2.7900    2.58925    2.62175  305252000.0
2004-08-23    2.76875    2.8370    2.72625    2.73500  365488000.0
2004-08-20    2.52525    2.7270    2.51250    2.70775  457144000.0
2004-08-19    2.50000    2.6015    2.39900    2.50850  894076000.0

[4499 rows x 5 columns]


In [ ]:
#删掉空值
def Stock_Price_LSTM_Data_Processing(df,mem_his_days,pre_days):
    
    df.dropna(inplace=True)
    df.sort_index(inplace=True)
    #label 将收盘价作为label  预测未来10天，将收盘价的这一列拿到做label，往上移动10天的数据 用前十天来预测
    #pre_days = 10
    df['label'] = df['Close'].shift(-pre_days)#拿来做预测
    from sklearn.preprocessing import StandardScaler
    #选用所有的行，取到最后的列label  这一步是进行标准化工作
    scaler = StandardScaler()
    sca_X = scaler.fit_transform(df.iloc[:,:-1])
    #print(sca_X)
    #mem_his_days = 10
    from collections import deque
    deq = deque(maxlen=mem_his_days)

    X = []
    for i in sca_X:
        deq.append(list(i))
        if len(deq)==mem_his_days:
            X.append(list(deq))

    X_lately = X[-pre_days:]
    X = X[:-pre_days]
   # print(len(X))#每一个deq里有5个，X最后将其全部append了
    #print(X)
    #print(len(X_lately))

    y = df['label'].values[mem_his_days-1:-pre_days]#TODO: -1有一点点疑惑，以后懂了再解决
    #print(len(y))
    import numpy as np
    X = np.array(X)
    y =np.array(y)
    print(X.shape)
    print(y.shape)
    return X,y,X_lately

In [ ]:
X,y,X_lately = Stock_Price_LSTM_Data_Processing(df,10,10)
print(X[0][1])
print(y)

In [ ]:
#优化一些模型，回溯 5 10 15天进行尝试 创造一些list
#创造一些lstm的记忆层，1,2,3,层
#dense  层 3层 时间运行长短的问题
#units 神经元每层有多少个
# 利用checkpoint设立检查点
pre_days = 10
mem_days = [5,10,15]
lstm_layers = [1,2,3]
dense_layers = [1,2,3]
units = [16,32]
from tensorflow.keras.callbacks import ModelCheckpoint
for the_mem_day in mem_days:
    for the_lstm_layer in lstm_layers:
        for the_dense_layer in dense_layers:
            for the_unit in units:
                filepath = './models/{val_mape:.2f}_{epoch:02d}_'+f'mem_{the_mem_day}_lstm_{the_lstm_layer}_dense_{the_dense_layer}_unit_{the_unit}'
                checkpoint = ModelCheckpoint(
                    filepath = filepath,
                    save_wieights_only=False,
                    monitor='val_mape',
                    mode='min',
                    save_bests_only=True
                )
                X,y,X_lately = Stock_Price_LSTM_Data_Processing(df,the_mem_day,pre_days)
                from sklearn.model_selection import train_test_split
                X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)
                #开始用tf进行预测了
                import tensorflow.keras as tf
                from tensorflow.keras.models import Sequential #引入神经网络中的层 dropout 删除一些值
                from tensorflow.keras.layers import LSTM,Dense,Dropout
                model = Sequential()

                model.add(LSTM(the_unit,input_shape=X.shape[1:],activation='relu',return_sequences=True))#1: 有点疑惑
                model.add(Dropout(0.1))
                for i in range(the_lstm_layer):

                    model.add(LSTM(the_unit,activation='relu',return_sequences=True))
                    model.add(Dropout(0.1))
                model.add(LSTM(the_unit,activation='relu'))
                model.add(Dropout(0.1))
                #10个神经单元 input_shape为X  等下会划分一下 防止过拟合，就删除一些神经元
                #可以构建一个全连接层，激活函数也是relu Dense 
                #做编译，有三个，优化器，损失函数，评价函数
                for i in range(the_dense_layer):

                    model.add(Dense(the_unit,activation='relu'))
                    model.add(Dropout(0.1))
                model.add(Dense(1))

                model.compile(optimizer='adam',
                             loss='mse',
                             metrics=['mape'])


                #开始训练
                model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

In [ ]:
from sklearn.preprocessing import StandardScaler
#选用所有的行，取到最后的列label  这一步是进行标准化工作
scaler = StandardScaler()
sca_X = scaler.fit_transform(df.iloc[:,:-1])
print(sca_X)
#损失是累加不是单词训练的结果，而是这一轮次内的
#实际的 x y test
#预测的predict
# best_model.evaluate(X_test,y_test)#实际
pre = best_model.predict(X_test)#预测

In [28]:
#优化一些模型，回溯 5 10 15天进行尝试 创造一些list
#创造一些lstm的记忆层，1,2,3,层
#dense  层 3层 时间运行长短的问题
#units 神经元每层有多少个
# 利用checkpoint设立检查点
pre_days = 10
mem_days = [5,10,15]
lstm_layers = [1,2,3]
dense_layers = [1,2,3]
units = [16,32]
from tensorflow.keras.callbacks import ModelCheckpoint
for the_mem_day in mem_days:
    for the_lstm_layer in lstm_layers:
        for the_dense_layer in dense_layers:
            for the_unit in units:
                filepath = './models/{val_mape:.2f}_{epoch:02d}_'+f'mem_{the_mem_day}_lstm_{the_lstm_layer}_dense_{the_dense_layer}_unit_{the_unit}'
                checkpoint = ModelCheckpoint(
                    filepath = filepath,
                    save_wieights_only=False,
                    monitor='val_mape',
                    mode='min',
                    save_bests_only=True
                )
                X,y,X_lately = Stock_Price_LSTM_Data_Processing(df,the_mem_day,pre_days)
                from sklearn.model_selection import train_test_split
                X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)
                #开始用tf进行预测了
                import tensorflow.keras as tf
                from tensorflow.keras.models import Sequential #引入神经网络中的层 dropout 删除一些值
                from tensorflow.keras.layers import LSTM,Dense,Dropout
                model = Sequential()

                model.add(LSTM(the_unit,input_shape=X.shape[1:],activation='relu',return_sequences=True))#1: 有点疑惑
                model.add(Dropout(0.1))
                for i in range(the_lstm_layer):

                    model.add(LSTM(the_unit,activation='relu',return_sequences=True))
                    model.add(Dropout(0.1))
                model.add(LSTM(the_unit,activation='relu'))
                model.add(Dropout(0.1))
                #10个神经单元 input_shape为X  等下会划分一下 防止过拟合，就删除一些神经元
                #可以构建一个全连接层，激活函数也是relu Dense 
                #做编译，有三个，优化器，损失函数，评价函数
                for i in range(the_dense_layer):

                    model.add(Dense(the_unit,activation='relu'))
                    model.add(Dropout(0.1))
                model.add(Dense(1))

                model.compile(optimizer='adam',
                             loss='mse',
                             metrics=['mape'])


                #开始训练
                model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

(4454, 5, 5)
(4454,)
Epoch 1/50
126/126 [==============================] - 4s 9ms/step - loss: 1671.1182 - mape: 86.1598 - val_loss: 166.9499 - val_mape: 42.5871
Epoch 2/50
126/126 [==============================] - 1s 6ms/step - loss: 167.0445 - mape: 29.0512 - val_loss: 36.5618 - val_mape: 16.3970
Epoch 3/50
126/126 [==============================] - 1s 6ms/step - loss: 134.2350 - mape: 23.6588 - val_loss: 20.6168 - val_mape: 12.6578
Epoch 4/50
126/126 [==============================] - 1s 6ms/step - loss: 109.3175 - mape: 22.3880 - val_loss: 11.0091 - val_mape: 11.5351
Epoch 5/50
126/126 [==============================] - 1s 6ms/step - loss: 114.9677 - mape: 21.3370 - val_loss: 8.6953 - val_mape: 9.9254
Epoch 6/50
126/126 [==============================] - 1s 6ms/step - loss: 113.9086 - mape: 20.5728 - val_loss: 14.1520 - val_mape: 9.9862
Epoch 7/50
126/126 [==============================] - 1s 6ms/step - loss: 105.5601 - mape: 19.7325 - val_loss: 10.6156 - val_mape: 8.6401
Epoch 8/

Epoch 10/50
125/125 [==============================] - 1s 8ms/step - loss: 44.5846 - mape: 14.2341 - val_loss: 11.6744 - val_mape: 7.8756
Epoch 11/50
125/125 [==============================] - 1s 7ms/step - loss: 44.3907 - mape: 13.4976 - val_loss: 8.8312 - val_mape: 7.5474
Epoch 12/50
125/125 [==============================] - 1s 7ms/step - loss: 47.5259 - mape: 13.9414 - val_loss: 17.0353 - val_mape: 10.2882
Epoch 13/50
125/125 [==============================] - 1s 8ms/step - loss: 41.2759 - mape: 12.7131 - val_loss: 15.4845 - val_mape: 8.1339
Epoch 14/50
125/125 [==============================] - 1s 7ms/step - loss: 38.9883 - mape: 12.9321 - val_loss: 9.4600 - val_mape: 6.5530
Epoch 15/50
125/125 [==============================] - 1s 8ms/step - loss: 38.9662 - mape: 12.6303 - val_loss: 9.6015 - val_mape: 7.6794
Epoch 16/50
125/125 [==============================] - 1s 7ms/step - loss: 44.9777 - mape: 14.1668 - val_loss: 14.9288 - val_mape: 7.2088
Epoch 17/50
125/125 [===============

125/125 [==============================] - 1s 7ms/step - loss: 55.6827 - mape: 14.6383 - val_loss: 112.4550 - val_mape: 13.8106
Epoch 20/50
125/125 [==============================] - 1s 7ms/step - loss: 57.4211 - mape: 14.0594 - val_loss: 85.5130 - val_mape: 12.7579
Epoch 21/50
125/125 [==============================] - 1s 6ms/step - loss: 53.6924 - mape: 14.2811 - val_loss: 128.1158 - val_mape: 17.3690
Epoch 22/50
125/125 [==============================] - 1s 6ms/step - loss: 59.1733 - mape: 14.3010 - val_loss: 78.2482 - val_mape: 10.5436
Epoch 23/50
125/125 [==============================] - 1s 6ms/step - loss: 56.6856 - mape: 13.7527 - val_loss: 132.6992 - val_mape: 15.4426
Epoch 24/50
125/125 [==============================] - 1s 6ms/step - loss: 56.7639 - mape: 14.1786 - val_loss: 60.0333 - val_mape: 10.6929
Epoch 25/50
125/125 [==============================] - 1s 7ms/step - loss: 46.8609 - mape: 13.7605 - val_loss: 73.5269 - val_mape: 11.5360
Epoch 26/50
125/125 [===============

KeyboardInterrupt: 

In [28]:
from sklearn.preprocessing import StandardScaler
#选用所有的行，取到最后的列label  这一步是进行标准化工作
scaler = StandardScaler()
sca_X = scaler.fit_transform(df.iloc[:,:-1])
print(sca_X)
#损失是累加不是单词训练的结果，而是这一轮次内的
#实际的 x y test
#预测的predict
# best_model.evaluate(X_test,y_test)#实际
pre = best_model.predict(X_test)#预测

[[-0.99286506 -0.99027401 -0.99559177 -0.99272414  4.99923362]
 [-0.99211526 -0.9865848  -0.99218442 -0.98680629  2.15136524]
 [-0.9848845  -0.98335122 -0.98576749 -0.98599695  1.55396284]
 ...
 [ 2.24544951  2.25971665  2.26087514  2.2503912  -0.66714129]
 [ 2.19936265  2.19292867  2.13709973  2.16904118 -0.54687189]
 [ 2.13798279  2.14477783  2.13649932  2.16235854 -0.59701651]]


In [36]:
#设置记忆的天数，假设记忆5天
#设置一个先进先出的队列  使得其等于要记忆的天数
#把deq append一下，当len长度为要记忆的天数的时候，那就append一个deq，超过30天，就先进先出
#把最后几条记录一下，跟pre_days有关系的那几条

#mem_his_days是打包5天的数据，跟预测多少天后没有关系，y要与X齐平所以要先砍掉；pre_days是预测多少天后的股票价格，y前面label值shift(-pre_days)后，
#最后pre_days是空值（你自己打印出来看一下），为了做曲线对比也要截取掉
#打包9.1-9.5预测的是9.15日的价格


#标准化是为了什么？不同特征之间的数值大小，要拉平，才能拟合出一个合适的模型权重
#特征的空间向量 构造的是X 接下来构造y  y和X长度要是一样的
mem_his_days = 10
from collections import deque
deq = deque(maxlen=mem_his_days)

X = []
for i in sca_X:
    deq.append(list(i))
    if len(deq)==mem_his_days:
        X.append(list(deq))
        
X_lately = X[-pre_days:]
X = X[:-pre_days]
print(len(X))#每一个deq里有5个，X最后将其全部append了
#print(X)
print(len(X_lately))

y = df['label'].values[mem_his_days-1:-pre_days]#TODO: -1有一点点疑惑，以后懂了再解决
print(len(y))

4479
10
4479


In [37]:
import numpy as np
X = np.array(X)
y =np.array(y)
print(X.shape)
print(y.shape)

(4479, 10, 5)
(4479,)


In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [ ]:
#开始用tf进行预测了
import tensorflow.keras as tf
from tensorflow.keras.models import Sequential #引入神经网络中的层 dropout 删除一些值
from tensorflow.keras.layers import LSTM,Dense,Dropout
model = Sequential()
model.add(LSTM(10,input_shape=X.shape[1:],activation='relu',return_sequences=True))#1: 有点疑惑
model.add(Dropout(0.1))

model.add(LSTM(10,activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(10,activation='relu'))
model.add(Dropout(0.1))
#10个神经单元 input_shape为X  等下会划分一下 防止过拟合，就删除一些神经元
#可以构建一个全连接层，激活函数也是relu Dense 
#做编译，有三个，优化器，损失函数，评价函数
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(optimizer='adam',
             loss='mse',
             metrics=['mape'])


#开始训练
model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 指定只是用第三块GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4416174819080898304
xla_global_id: -1
]


In [4]:
import tensorflow as tf1
print("Num GPUs Available: ", len(tf1.config.experimental.list_physical_devices('GPU')))
print("Num GPUs Available: ", len(tf1.config.experimental.list_physical_devices('GPU')))
print(tf1.test.is_gpu_available())


Num GPUs Available:  0
Num GPUs Available:  0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


In [ ]:
import tensorflow as tf1
with tf1.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf1.Session() as sess:
    print (sess.run(c))


In [3]:
import tensorflow as tf
print("Version: " + tf.__version__)


Version: 2.7.3


In [55]:
import tensorflow as tf1
print("Num GPUs Available: ", len(tf1.config.experimental.list_physical_devices('GPU')))print("Num GPUs Available: ", len(tf1.config.experimental.list_physical_devices('GPU')))



SyntaxError: invalid syntax (3570505637.py, line 2)

In [46]:
import tensorflow as tf1
with tf1.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf1.Session() as sess:
    print (sess.run(c))


AttributeError: module 'tensorflow' has no attribute 'Session'

In [56]:
import tensorflow as tf
print("Version: " + tf.__version__)


Version: 2.7.3
